Cluster level analysis: 


consider CDF, Histogram
y: % of unprod. reads / total cluster reads
x: 
    - intron length, total span of cluster?
    - number of introns in cluster
    - number of PTCs in cluster normalized by cluster length

might need to remove alternative TSS

Gene level analysis:
    

In [3]:
suppressMessages(library(tidyverse))
suppressMessages(library(glue))
suppressMessages(library(data.table))

In [4]:
suppressMessages(library(cowplot))
options(repr.plot.width=10, repr.plot.height=5, repr.plot.res = 180)
theme_set(theme_cowplot())

In [5]:
suppressMessages(library(GenomicRanges))

In [6]:
setwd("~/cdai/SpliFi/analysis")

In [7]:
suppressMessages(library(furrr))
plan(multisession, workers = 6)

In [8]:
# counts for leafcutter2 output, include consititutive introns

eur  <- readRDS('../data/ExtractFractions/Geuvadis/EUR.numerators_constcounts.noise_by_intron.rds')

In [9]:
dim(eur)
eur[1:5, c(1:5, 375:377)]

[1] 228671    377

chrom,clu,clu_type,intron_type,HG00096,NA20819,NA20826,NA20828
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
chr1:779092:803918:clu_1_+:N,clu_1,N,N,0,0,2,0
chr1:804222:804775:clu_2_+:N,clu_2,N,N,0,0,2,1
chr1:804222:807216:clu_2_+:N,clu_2,N,N,0,0,0,0
chr1:807323:809657:clu_3_+:N,clu_3,N,N,0,0,0,0
chr1:827775:829002:clu_4_+:N,clu_4,N,N,12,12,14,25


In [10]:
data_cols = names(eur)[str_detect(names(eur), '[\\w]+\\d+')]

In [11]:
 eur.meds <- eur[, ..data_cols] %>% as.matrix %>% matrixStats::rowMedians()

In [12]:
eur.meds <- eur[, .(chrom, clu, clu_type, intron_type, meds = eur.meds)]

In [13]:
eur.meds[1:10,]

chrom,clu,clu_type,intron_type,meds
<chr>,<chr>,<chr>,<chr>,<dbl>
chr1:779092:803918:clu_1_+:N,clu_1,N,N,0
chr1:804222:804775:clu_2_+:N,clu_2,N,N,0
chr1:804222:807216:clu_2_+:N,clu_2,N,N,0
chr1:807323:809657:clu_3_+:N,clu_3,N,N,0
chr1:827775:829002:clu_4_+:N,clu_4,N,N,16
chr1:827849:829002:clu_4_+:N,clu_4,N,N,1
chr1:827853:829002:clu_4_+:N,clu_4,N,N,1
chr1:829104:841199:clu_4_+:N,clu_4,N,N,4
chr1:829104:847653:clu_4_+:N,clu_4,N,N,7


In [14]:
eur.meds <- eur.meds[, .(clu, intron_type, meds)][, .(meds = sum(meds)), by = .(clu, intron_type)]

In [15]:
computePratio  <- function(dt) {
    if ('F' %in% dt$intron_type & 'N' %in% dt$intron_type) {
        dt$Pratio <- dt[dt$intron_type == 'F']$meds / sum(dt$meds)
    } else if ('F' %in% dt$intron_type) {
        dt$Pratio  <- 1
    } else {
        dt$Pratio <- 0
    }

    return(dt)
}

In [16]:
computePratio(eur.meds[clu == 'clu_5'])

clu,intron_type,meds,Pratio
<chr>,<chr>,<dbl>,<dbl>
clu_5,N,0,0


In [17]:
Pratio <- split(eur.meds, by = 'clu') %>% map(computePratio)

In [18]:
Pratio  <- rbindlist(Pratio)

In [19]:
Pratio <- Pratio[!is.na(Pratio), .(clu, Pratio)] %>% unique

In [21]:
Pratio[1:5]

clu,Pratio
<chr>,<dbl>
clu_1,0
clu_2,0
clu_3,0
clu_4,0
clu_5,0


In [22]:

chrom_splits  <- str_split(eur$chrom, ":")
chrom_splits[1:2]

[[1]]
[1] "chr1"    "779092"  "803918"  "clu_1_+" "N"      

[[2]]
[1] "chr1"    "804222"  "804775"  "clu_2_+" "N"

In [38]:
map_chr(chrom_splits, ~.x[[3]])

[1] "803918"    "804775"    "807216"    "809657"    "829002"    "829002"   
    [7] "829002"    "841199"    "847653"    "851926"    "847653"    "847653"   
   [13] "849483"    "850177"    "850180"    "851926"    "850177"    "850180"   
   [19] "851926"    "851926"    "852670"    "853390"    "853390"    "856448"   
   [25] "961292"    "961628"    "961825"    "962285"    "962354"    "962703"   
   [31] "963031"    "963108"    "963336"    "963919"    "964348"    "964348"   
   [37] "964934"    "964962"    "966703"    "970276"    "970520"    "970685"   
   [43] "970878"    "971076"    "971323"    "972074"    "972287"    "972860"   
   [49] "973185"    "973499"    "974315"    "974441"    "1013983"   "1013983"  
   [55] "1014004"   "1013983"   "1035276"   "1035276"   "1040664"   "1040664"  
   [61] "1043537"   "1041477"   "1041955"   "1044333"   "1045963"   "1046396"  
   [67] "1046396"   "1046819"   "1047326"   "1047572"   "1047775"   "1048011"  
   [73] "1048866"   "1049235"   "1049565"   "1049902"   "1050232"   "1050426"  
   [79] "1050725"   "1051031"   "1051252"   "1051252"   "1051727"   "1053724"  
   [85] "1053752"   "1053752"   "1054823"   "1055065"   "1055376"   "1055376"  
   [91] "1055897"   "1060276"   "1060279"   "1060276"   "1060279"   "1061019"  
   [97] "1063078"   "1065829"   "1063078"   "1065829"   "1065829"   "1065351"  
  [103] "1065829"   "1065833"   "1065948"   "1099176"   "1099717"   "1279067"  
  [109] "1279330"   "1283977"   "1285925"   "1286767"   "1287108"   "1287507"  
  [115] "1287672"   "1287938"   "1290270"   "1290476"   "1290636"   "1290894"  
  [121] "1291064"   "1291253"   "1308914"   "1309085"   "1309085"   "1309453"  
  [127] "1309680"   "1309705"   "1310633"   "1310891"   "1310908"   "1311329"  
  [133] "1326835"   "1327240"   "1327240"   "1331637"   "1332023"   "1332920"  
  [139] "1355262"   "1400157"   "1399978"   "1400157"   "1400604"   "1400604"  
  [145] "1406932"   "1406932"   "1436926"   "1437321"   "1439080"   "1454344"  
  [151] "1456224"   "1457128"   "1477273"   "1477750"   "1478643"   "1478643"  
  [157] "1516011"   "1516011"   "1516011"   "1480866"   "1482137"   "1489691"  
  [163] "1490256"   "1490256"   "1485781"   "1486109"   "1486543"   "1487862"  
  [169] "1489203"   "1489203"   "1518920"   "1520140"   "1520547"   "1522743"  
  [175] "1522743"   "1527160"   "1527694"   "1527694"   "1529222"   "1529222"  
  [181] "1533925"   "1616507"   "1616507"   "1617077"   "1623388"   "1623430"  
  [187] "1623773"   "1623773"   "1623773"   "1624964"   "1624990"   "1625285"  
  [193] "1625285"   "1629132"   "1629384"   "1629638"   "1629638"   "1625545"  
  [199] "1625557"   "1626649"   "1625908"   "1626649"   "1626836"   "1627073"  
  [205] "1627672"   "1628018"   "1628272"   "1628488"   "1630291"   "1633187"  
  [211] "1633988"   "1634201"   "1662330"   "1751723"   "1699941"   "1700139"  
  [217] "1700346"   "1701147"   "1701360"   "1701607"   "1916973"   "1917146"  
  [223] "2055440"   "2056483"   "2059540"   "2135261"   "2135261"   "2144209"  
  [229] "2146026"   "2148871"   "2169517"   "2184582"   "2172054"   "2172300"  
  [235] "2173896"   "2174753"   "2184921"   "2302977"   "2303284"   "2303839"  
  [241] "2304292"   "2304292"   "2318989"   "2306576"   "2393207"   "2393269"  
  [247] "2394646"   "2395783"   "2395783"   "2395783"   "2395783"   "2394646"  
  [253] "2395783"   "2395783"   "2395783"   "2399414"   "2400856"   "2402206"  
  [259] "2400856"   "2402206"   "2402096"   "2402206"   "2425708"   "2430337"  
  [265] "2478475"   "2479733"   "2480182"   "2484447"   "2487597"   "2489206"  
  [271] "2494855"   "2496606"   "2495487"   "2496827"   "2497501"   "2498522"  
  [277] "2498522"   "2498743"   "2499083"   "2499640"   "2502111"   "2503921"  
  [283] "2550963"   "2552002"   "2556584"   "2556584"   "2556584"   "2556584"  
  [289] "2557725"   "2557725"   "2557725"   "2557725"   "2557725"   "2558619"  
  [295] "2558687"   "2558880"   "2559659"   "2559822"   "2559908"   "2561493"  
  [3

In [75]:
introns <- data.frame(
    seqnames = map_chr(chrom_splits, ~.x[[1]]),
    start = map_chr(chrom_splits, ~.x[[2]]),
    end = map_chr(chrom_splits, ~.x[[3]]),
    strand = map(chrom_splits, ~str_split(.x[[4]], "_")) %>% map(~.x[[1]][[3]])
)

: 

: 

In [ ]:
introns %>% head

,seqnames,start,end,strand
,<chr>,<chr>,<chr>,<chr>
1,chr1,779092,803918,clu_1_+
2,chr1,804222,804775,clu_2_+
3,chr1,804222,807216,clu_2_+
4,chr1,807323,809657,clu_3_+
5,chr1,827775,829002,clu_4_+
6,chr1,827849,829002,clu_4_+


In [74]:
map(chrom_splits[1:5], ~str_split(.x[[4]], "_")) %>% map(~.x[[1]][[3]])

[[1]]
[1] "+"

[[2]]
[1] "+"

[[3]]
[1] "+"

[[4]]
[1] "+"

[[5]]
[1] "+"